In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from iso3166 import countries

import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import dash_bootstrap_components as dbc

In [2]:
all_trans = pd.read_excel("data/transcripts.xlsx", index_col=0)
weights=pd.read_excel("data/lda_df_weights_update.xlsx", index_col=0)
df_merged=weights
weights.head()

,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,year,prevalent,country
0,0.082801,0.432069,0.028909,0.264858,0.003181,0.188182,1970,Topic 2,ALB
1,0.525291,0.457330,0.001100,0.009987,0.006222,0.000070,1970,Topic 1,ARG
2,0.360016,0.454747,0.004544,0.004296,0.137150,0.039247,1970,Topic 2,AUS
3,0.648231,0.312949,0.000106,0.027700,0.006662,0.004351,1970,Topic 1,AUT
4,0.582216,0.397412,0.000110,0.002327,0.007637,0.010299,1970,Topic 1,BEL


In [9]:
countries = all_trans.country.unique()
topic_list={"International Diplomacy":"Topic 1","War and Power":"Topic 2","Conflict in Africa":"Topic 3","Middle Eastern Terrorism":"Topic 4","Climate Change":"Topic 5","Communism":"Topic 6"}

In [20]:
{'label': i, 'value': i} for i in topic_list.keys()

SyntaxError: invalid syntax (<ipython-input-20-689a8f572843>, line 1)

In [22]:
# Build App
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP], suppress_callback_exceptions=True)

app.layout = html.Div([
    
    html.Div([
        html.H1("Modern Data Analytics Project"),
        html.H2("Politicians and Climate Change"),
        html.Div("This Dashboard displays and summarizes some UN-speeches"),
        
        html.Br(),
        
        dcc.Dropdown(
            id='country_dropdown',
            options=[
                {'label': i, 'value': i} for i in countries
            ],
            value = 'ALB',
            placeholder='Select a country',
            style={'width': '100%'}
        ),
        
        html.Br(),
        
        html.Div([
        #dcc.Input(id='term_select', type='text', debounce=False,
        #         value='climate', required=True),
        dcc.Dropdown(
            id='topic_select',
            options=[
                {'label': i, 'value': i} for i in topic_list.keys()
            ],
            value = 'Climate Change',
            placeholder='Choose a Topic',
            style={'width': '100%'}
        ),
    ]),
    
    ]),

    html.Div(id='text_out', children=[]),
    
    html.Div([
        dcc.Graph(id='country_graph', figure={})
    ]),

])

@app.callback(
    [Output(component_id='text_out', component_property='children')],
     Output(component_id='country_graph', component_property='figure'),     
    Input(component_id='country_dropdown', component_property='value'),
    Input(component_id='topic_select', component_property='value')
)

def update_graph(my_dropdown, topic_select):
    
    dfff = df_merged
    data_filtered = dfff.loc[dfff['country'] == my_dropdown]
    
    year_fill = np.array(data_filtered["year"].unique())
    weights=np.array(data_filtered[topic_list[topic_select]])

    
    scatter=px.scatter(
        x=year_fill,
        y=weights,
        trendline="lowess",
        ).update_xaxes(range=[min(df_merged["year"]),max(df_merged["year"])])
    
    container = 'Number of speeches containing the term "{}" given by {}'.format(topic_select, my_dropdown)
    
    return container, scatter

app.run_server(mode='jupyterlab', debug=False,dev_tools_ui=False,dev_tools_props_check=False)

127.0.0.1 - - [27/May/2021 20:04:54] "GET /_shutdown_1d976d36-2887-4498-a369-cf5115668fa5 HTTP/1.1" 200 -
 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [27/May/2021 20:04:54] "GET /_alive_1d976d36-2887-4498-a369-cf5115668fa5 HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2021 20:04:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2021 20:04:54] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2021 20:04:54] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2021 20:04:54] "POST /_dash-update-component HTTP/1.1" 200 -
